<h2 align="center">
	Unitary Preparation of Tensor Network States
</h2>

In [1]:
import numpy as np
import pickle as pkl

import quimb.tensor as qtn

from qsp.misc_states import make_aklt_mps
from qsp.misc_states import make_aklt_peps
from qsp.tsp import MPSPreparation, PEPSPreparation
from qsp.tsp_helper_routines import make_splitted_mps

Given the requirements load/construct MPS of cetain length.

In [5]:
mps_type = 'aklt'#'random'#'N2'#'P4'#'heisenberg'#

if mps_type == 'aklt':
    tensor_array, _ = make_aklt_mps(L=6)
    tensor_array = make_splitted_mps(tensor_array)
    target_mps = qtn.MatrixProductState(tensor_array, shape='lrp')


if mps_type == 'random':
    target_mps = qtn.MPS_rand_state(L=12, bond_dim=4)
    target_mps.permute_arrays(shape='lrp')
    
    
if mps_type in ['P4','N2','heisenberg']:
    filenames = {'P4': 'data/P4_6-31G_dist2.0000.pkl',              # 
                    'N2': 'data/N2_STO-6G_dist2.0000.pkl',
                    'heisenberg':'data/heisenberg_L32_dist0.8000.pkl'}
    with open(filenames[mps_type], 'rb') as f:
        data = pkl.load(f)
    target_mps = data['quimb_mps']

<h2 align="center">
	Matrix Product States
</h2>
<h3 align="left">
	1. Sequential preparation
</h3>


$$
| \psi[A] \rangle \approx  (U_0. U_1. … U_D) |00...0\rangle
$$


where $D$ is the number of layers of unitaries $U_i$ (see [[1]], [[2]]).
 

[1]: https://arxiv.org/pdf/2209.07106.pdf
[2]: https://journals.aps.org/pra/abstract/10.1103/PhysRevA.101.032310

In [6]:
prep = MPSPreparation(target_mps)
number_of_layers = 6
prep.sequential_unitary_circuit(number_of_layers, do_compression=False)

preparing mps using sequential unitaries...


100%|██████████| 6/6 [00:01<00:00,  3.93it/s]


overlap from static seq. preparation = 0.94190485 (from circ 0.94190491),
n_gates=453, n_2qg=147



In [7]:
circ = prep.seq_data['circ']
circ.size()

453

<h3 align="left">
	2. Variational optimization over sequentially prepared unitaries
</h3>

In [8]:
prep = MPSPreparation(target_mps)
number_of_layers = 2
n_iter, nhop = 40, 4
prep.sequential_unitary_circuit_optimization(number_of_layers, do_compression=False, n_iter=n_iter, nhop=nhop)

doing variational optimization over sequential unitaries...


100%|██████████| 2/2 [00:00<00:00, 11.86it/s]


number of variational params in the circuit (from sequential algorithm) are 483
overlap before variational optimization = -0.8447832465


-0.906573712826 [best: -0.906573712826] :  10%|█         | 16/160 [00:40<06:30,  2.72s/it]

<h3 align="left">
	3. Quantum Circuit Tensor Network Ansatz
</h3>

In [ ]:
prep = MPSPreparation(target_mps)
depth = 8
n_iter, nhop = 40, 4,
prep.quantum_circuit_tensor_network_ansatz(depth, n_iter=n_iter, nhop=nhop)

<h3 align="left">
	4. Preparation using Linear Combinition of unitaries (LCU)
</h3>

\begin{aligned}
	|\psi[A] \rangle = \left( U_0 + \kappa_1U_1 + \kappa_2U_2 + \dots\right)|00\dots0\rangle,
\end{aligned}

see [[1]] for details

[1]: https://arxiv.org/pdf/2209.07106.pdf


In [ ]:
prep = MPSPreparation(target_mps)
number_of_lcu_layers = 4
prep.lcu_unitary_circuit(number_of_lcu_layers, verbose=False)

<h3 align="left">
	5. Variational optimization over LCU construction
</h3>

In [ ]:
prep = MPSPreparation(target_mps)
number_of_lcu_layers = 4
prep.lcu_unitary_circuit_optimization(number_of_lcu_layers, verbose=True)

<h3 align="left">
	6. Adiabatic state Preparation of MPS
</h3>

In [ ]:
target_mps = qtn.MPS_rand_state(L=6, bond_dim=2)
prep = MPSPreparation(target_mps)
Tmax, tau = 8, 0.04 #total runtime, trotter step size
max_bond = 2
prep.adiabatic_state_preparation(Tmax, tau, max_bond, verbose=False)

plt.plot(mps_p.adiabatic_data['target_fidelity'].keys(), mps_p.adiabatic_data['target_fidelity'].values(), '.-')
plt.title('random mps')
plt.xlabel('time')
plt.ylabel('current fidelity')

In [ ]:
tensor_array, _ = make_aklt_mps(L=6)
prep = MPSPreparation(tensor_array, shape='lrp')
Tmax, tau = 8, 0.04 # total runtime, trotter step size
max_bond = 2
prep.adiabatic_state_preparation(Tmax, tau, max_bond, verbose=False)    
plt.plot(prep.adiabatic_data['target_fidelity'].keys(), prep.adiabatic_data['target_fidelity'].values(), '.-')
plt.title('1d aklt')
plt.xlabel('time')
plt.ylabel('current fidelity')

<h2 align="center">
	Projected Entangled Pair States (PEPS) in 2d
</h2>
<h3 align="left">
	1. Adiabatic state Preparation
</h3>

In [ ]:
Lx, Ly = 10, 4
target_grid, _ = make_aklt_peps(Lx, Ly)
prep = PEPSPreparation(target_grid)

Tmax, tau = 10, 0.04
max_bond = 2
prep.adiabatic_state_preparation(Tmax, tau, max_bond, verbose=False)

plt.plot(prep.adiabatic_data['target_fidelity'].keys(), prep.adiabatic_data['target_fidelity'].values(), '.-')